In [1]:
import numpy as np
import pandas as pd

In [2]:
movies_df = pd.read_csv('movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
rating_df=pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [3]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
df = pd.merge(rating_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
combine_movie_rating = df.dropna(axis=0,subset=['title'])
movie_rating = combine_movie_rating.groupby(by=['title'])['rating'].count().reset_index().rename(columns={'rating':'totalratingCount'})[['title','totalratingCount']]
movie_rating.head()

,title,totalratingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [7]:
combine_movie_rating.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [8]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_rating, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,title,totalratingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [9]:
# a = ['ytKveh',4]
# s1 = a[0].lower()
# result = ''
# diff = a[1]
# for i in s1:
#     val = ord(i)
#     if val-diff < 97:
#         x = 97 - val
#         y = 122-x
#         result += chr(y)
#     else:
#         result += chr(val-diff)

# final_result = ''
# for i in range(len(a[0])):
#     if(a[0][i].isupper()):
#         final_result += result[i].upper()
#     else:
#         final_result += result[i]

# print(final_result)

In [10]:
popularity_threshold = 50
popular_movies = rating_with_totalRatingCount.query('totalratingCount >= @popularity_threshold')
popular_movies.head()

,userId,movieId,rating,title,totalratingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [11]:
popular_movies.shape

(41362, 5)

In [12]:
movie_features = popular_movies.pivot_table(index = 'title',columns='userId',values='rating').fillna(0)

In [13]:
movie_features.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [14]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features)

from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
movie_features.shape

(450, 606)

In [16]:
query_index = np.random.choice(movie_features.shape[0])
print(query_index)

205


In [17]:
distances, indicies = model_knn.kneighbors(movie_features.iloc[query_index,:].values.reshape(1,-1), n_neighbors=6)

In [20]:
for i in range(0,len(distances.flatten())):
    if i == 0:
        print(f'Recommendations for {movie_features.index[query_index]} \n')
    else:
        print(f"{i}: {movie_features.index[indicies.flatten()[i]]} , with distance {distances.flatten()[i]}")

Recommendations for Hot Shots! Part Deux (1993) 

1: Naked Gun 33 1/3: The Final Insult (1994) , with distance 0.43138933181762695
2: Last Action Hero (1993) , with distance 0.538442850112915
3: Robin Hood: Men in Tights (1993) , with distance 0.5510671138763428
4: Mars Attacks! (1996) , with distance 0.5652288198471069
5: Beverly Hills Cop III (1994) , with distance 0.5897572040557861


In [22]:
def random_prec(n):
    query_index = np.random.choice(movie_features.shape[0])
    distances, indicies = model_knn.kneighbors(movie_features.iloc[query_index,:].values.reshape(1,-1), n_neighbors=n)
    for i in range(0,len(distances.flatten())):
        if i == 0:
            print(f'Recommendations for {movie_features.index[query_index]} \n')
        else:
            print(f"{i}: {movie_features.index[indicies.flatten()[i]]} , with distance {distances.flatten()[i]}")

In [28]:
random_prec(2)

Recommendations for Spider-Man (2002) 

1: Spider-Man 2 (2004) , with distance 0.26784181594848633
